In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [ ]:
# Initialize the Chrome driver
driver = webdriver.Chrome()


In [ ]:
driver.get('https://www.amazon.in/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.in%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=inflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0')

In [ ]:
# Function to log in to Amazon
def login_to_amazon(phone_number, password):
    driver.get('https://www.amazon.in/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.in%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=inflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0')
    
    # Wait for the phone number field to load
    phone_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'ap_email'))
    )
    phone_field.send_keys(phone_number)
    phone_field.send_keys(Keys.RETURN)
    
    # Wait for the password field to load
    password_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'ap_password'))
    )
    password_field.send_keys(password)
    password_field.send_keys(Keys.RETURN)
    
    # Wait for login to complete
    time.sleep(5)  # Adjust sleep time as needed

In [ ]:

# Replace with your Amazon credentials
PHONE_NUMBER = '7820983685'  # Replace with your Amazon phone number
PASSWORD = 'nagesh09876'  # Replace with your Amazon password


In [ ]:
# calling the function:
login_to_amazon(PHONE_NUMBER, PASSWORD)

In [ ]:
# Navigating to the product reviews page
driver.get('https://www.amazon.in/Apple-iPhone-15-128-GB/product-reviews/B0CHX3TW6X/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

# NOW SCRAPPING REVIVES

In [ ]:
# an empty DataFrame to store all reviews
all_reviews_df = pd.DataFrame(columns=['Review'])

In [ ]:
# Loop through all pages of reviews
while True:
    # Parse the page source using BeautifulSoup
    html_data = BeautifulSoup(driver.page_source, "html.parser")
    
    # Extract reviews
    reviews = html_data.find_all('div', {'class': 'a-row a-spacing-small review-data'})
    
    # Store in a list
    review_texts = []
    
    for review in reviews:
        span_tag = review.find('span', {'class': 'a-size-base'})
        if span_tag:
            review_texts.append(span_tag.text.strip())
    
    # Convert to DataFrame
    page_reviews_df = pd.DataFrame(review_texts, columns=['Review'])
    
    # Append to the main DataFrame
    all_reviews_df = pd.concat([all_reviews_df, page_reviews_df], ignore_index=True)
    
    # Check if there is a "Next Page" button
    try:
        next_page_button = driver.find_element(By.CLASS_NAME, 'a-last')
        if 'a-disabled' in next_page_button.get_attribute('class'):
            break  # Exit the loop if there is no next page
        next_page_button.click()
        
        # Wait for the next page to load
        time.sleep(5)  # Adjust sleep time as needed
    except Exception as e:
        print("No more pages or error navigating to the next page:", e)
        break

# Close the driver
driver.quit()

# Display the final DataFrame
print(all_reviews_df)

# Save the DataFrame to a CSV file (optional)
all_reviews_df.to_csv('amazon_reviews.csv', index=False)

                                               Review
0   So lightweight. battery life is good .better s...
1   DesignOne of the most noticeable changes to th...
2   It’s an amazing product iphone 15 bought for m...
3   The media could not be loaded.\n              ...
4   Just made the transition and here’s my POV on ...
..                                                ...
95                                  It’s iPhone man …
96  1. Good display -• Peak brightness is good and...
97  At starting am hesitating to buy this much cos...
98  Product deliver safely and ok but invoice and ...
99  I've switched from iPhone 13 to 15 and this tr...

[100 rows x 1 columns]


In [2]:
all_reviews_df

,Review
0,So lightweight. battery life is good .better s...
1,DesignOne of the most noticeable changes to th...
2,It’s an amazing product iphone 15 bought for m...
3,The media could not be loaded.\n ...
4,Just made the transition and here’s my POV on ...
...,...
95,It’s iPhone man …
96,1. Good display -• Peak brightness is good and...
97,At starting am hesitating to buy this much cos...
98,Product deliver safely and ok but invoice and ...


# DATA PREPROCESSING :